In [1]:
#inicjalizacja bibliotek
import numpy as np
import pandas as pd
from sympy import symbols, Eq, solve

import copy #creates deepcopy

#GUI
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from tkinter.ttk import Combobox
from tkinter import filedialog as fd

#Figures
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, NavigationToolbar2Tk)
from matplotlib.backends.backend_pdf import PdfPages #zapis do pdfu z listy figs

#Optimization
from scipy.optimize import curve_fit

## Parts related to Concentration_distribution and Concentration_fit 

In [2]:
# this class relates to 
class concentration:
    def __init__(self, Element1, Element2, concentration_Element1, total_thickness, AK_Height, xsize, ysize, position, Element3, thickness_Element3):
        
        self.mol_data = {"Co":6.67*10**-6, "Ni":6.59*10**-6, "Au":10.21*10**-6, "Pt":9.09*10**-6, "Tb":1.93*10**-5} # dict with molecular volume of elements
        #self.mol_data = dict(pd.read_excel("molecular_database.xlsx"))
        
        self.Element1 = str(Element1)
        self.Element2 = str(Element2)
        self.concentration_Element1 = float(concentration_Element1)
        self.total_thickness = float(total_thickness)
        self.AK_Height = str(AK_Height)
        self.position = float(position) #control position of the sample center
        self.load_polynomial(self.AK_Height)
        self.xsize = float(xsize)
        self.ysize = float(ysize)

        self.thickness_Element3 = float(thickness_Element3)
        self.Element3 = str(Element3)

        self.epsilon_calibration()        
        
    def epsilon_calibration(self):
        
        tElement1, tElement2 = symbols('tElement1, tElement2')
        eq1 = Eq(100*(tElement1/self.mol_data[self.Element1])/((tElement1/self.mol_data[self.Element1]) + ((tElement2)/self.mol_data[self.Element2]) + ((self.thickness_Element3)/self.mol_data[self.Element3])),self.concentration_Element1)
        eq2 =  Eq(tElement2 + tElement1,self.total_thickness)
        solved = dict((str(key), float(value)) for (key, value) in solve((eq1,eq2),(tElement1,tElement2)).items())
        self.thickness_Element1  = solved['tElement1'] 
        self.thickness_Element2 = solved['tElement2']
        
        self.epsilon_Element1 = self.thickness_Element1/self.polynomial(65)
        self.epsilon_Element2 = self.thickness_Element2/self.polynomial(65)
        
    def load_polynomial(self,filename_poly):
        self.polynomial = np.poly1d(np.load(filename_poly))
        
    def calculate_concentration_1D(self):
        
        self.number_points = 201
        xt = 65
        xt2 = -65
        self.thickness_distribution_Element1 = np.zeros([self.number_points])
        self.thickness_distribution_Element2 = np.zeros([self.number_points])

        
        for lx,x in enumerate(np.linspace(-0.5*self.xsize,0.5*self.xsize,self.number_points)):
            ro = np.sqrt((x - xt+self.position)**2)
            ro2 = np.sqrt((x - xt2-self.position)**2)
            self.thickness_distribution_Element1[lx]=self.epsilon_Element1*self.polynomial(ro)
            self.thickness_distribution_Element2[lx]=self.epsilon_Element2*self.polynomial(ro2)
        self.total_thickness_distribution = self.thickness_distribution_Element1+self.thickness_distribution_Element2
        self.composition()    
    
    def calculate_concentration_2D(self):  
        self.number_points = 201
        xt = 65
        yt = 0
        xt2 = -65
        yt2 = 0
        self.thickness_distribution_Element1 = np.zeros([self.number_points,self.number_points])
        self.thickness_distribution_Element2 = np.zeros([self.number_points,self.number_points])

        
        for lx,x in enumerate(np.linspace(-0.5*self.xsize,0.5*self.xsize,self.number_points)):
            for ly,y in enumerate(np.linspace(-0.5*self.ysize,0.5*self.ysize,self.number_points)):
                ro = np.sqrt((x - xt+self.position)**2 + (y - yt)**2)
                ro2 = np.sqrt((x - xt2-self.position)**2 + (y - yt2)**2)
                self.thickness_distribution_Element1[lx,ly]=self.epsilon_Element1*self.polynomial(ro)
                self.thickness_distribution_Element2[lx,ly]=self.epsilon_Element2*self.polynomial(ro2)
        self.total_thickness_distribution = self.thickness_distribution_Element1+self.thickness_distribution_Element2 + self.thickness_Element3  
        self.composition()
        
    def composition(self):
        self.composition_distribution_Element1=100*self.thickness_distribution_Element1/self.mol_data[self.Element1]/(self.thickness_distribution_Element1/self.mol_data[self.Element1]+self.thickness_distribution_Element2/self.mol_data[self.Element2] + ((self.thickness_Element3)/self.mol_data[self.Element3]))
              
    
    def transform(self): #funckja zmieniająca tablice numpy na DataFrame
        def df_transform(data):
            df_data = pd.DataFrame(data).T
            df_data.index = np.linspace(0,self.xsize,self.number_points)
            df_data.columns = np.linspace(0,self.ysize,self.number_points)
            return df_data
        
        self.composition_distribution_Element1 = df_transform(self.composition_distribution_Element1)
        self.total_thickness_distribution = df_transform(self.total_thickness_distribution)
        self.thickness_distribution_Element1 = df_transform(self.thickness_distribution_Element1)
        self.thickness_distribution_Element2 = df_transform(self.thickness_distribution_Element2)      
             
        
    def draw_figures_2D(self):
        
        self.transform()
              
        font=12
        plt.figure(0)
        plt.clf()
        data = self.composition_distribution_Element1
        plt.imshow(data, cmap='nipy_spectral',interpolation='gaussian',extent=[data.columns[0],data.columns[-1],data.columns[0],data.columns[-1]])
        plt.title(f"Concentration distribution of {self.Element1} [at.%]",fontsize=font)
        plt.xlabel("X position [mm]",fontsize=font)
        plt.ylabel("Y position [mm]",fontsize=font)
        plt.minorticks_on()
        plt.tick_params(axis="x", which = "major", length=10, direction ="in",labelsize=10)
        plt.tick_params(axis="y", which = "major", length=10, direction ="in",labelsize=10, color = "w")
        plt.tick_params(axis="x", which = "minor", length=5, direction ="in")
        plt.tick_params(axis="y", which = "minor", length=5, direction ="in", color ="w")        
        plt.colorbar()
        plt.axhline(self.ysize/2, linestyle ="--",color="black",linewidth=3)
        
        plt.figure(1)
        plt.clf()
        data = self.composition_distribution_Element1
        plt.plot(data.loc[data.index==0].T)
        plt.title(f"Concentration distribution of {self.Element1}",fontsize=font)
        plt.xlabel("X position [mm]",fontsize=font)
        plt.ylabel(f"{self.Element1} concentration [at.%]",fontsize=font)
        plt.minorticks_on()
        plt.tick_params(axis="both", which = "major", length=10, direction ="in",labelsize=10)
        plt.tick_params(axis="both", which = "minor", length=5, direction ="in")
        
        plt.figure(2)
        plt.clf()
        data = self.total_thickness_distribution
        plt.imshow(data, cmap='nipy_spectral',interpolation='gaussian',extent=[data.columns[0],data.columns[-1],data.columns[0],data.columns[-1]])
        plt.title(f"Total thickness distribution of {self.Element1}{self.Element2} alloy [A]",fontsize=font)
        plt.xlabel("X position [mm]",fontsize=font)
        plt.ylabel("Y position [mm]",fontsize=font)
        plt.minorticks_on()
        plt.tick_params(axis="both", which = "major", length=10, direction ="in",labelsize=10)
        plt.tick_params(axis="both", which = "minor", length=5, direction ="in")
        plt.colorbar()
        plt.axhline(self.ysize/2, linestyle ="--",color="black",linewidth=3)
        
        plt.figure(3)
        plt.clf()
        data = self.total_thickness_distribution
        plt.plot(data.loc[data.index==0].T)
        plt.title(f"Total thickness distribution of {self.Element1}{self.Element2} alloy",fontsize=font)
        plt.xlabel("X position [mm]",fontsize=font)
        plt.ylabel("Total thickness [A]",fontsize=font)
        plt.minorticks_on()
        plt.tick_params(axis="both", which = "major", length=10, direction ="in",labelsize=10)
        plt.tick_params(axis="both", which = "minor", length=5, direction ="in")     
            
        self.fig = list(map(plt.figure, plt.get_fignums()))    
        
    def save_2D(self, file_dir):
        writer = pd.ExcelWriter(path = file_dir, engine='xlsxwriter')
        self.composition_distribution_Element1.to_excel(writer, sheet_name=f'composition_distribution_{self.Element1}')
        self.total_thickness_distribution.to_excel(writer, sheet_name='total_thickness_distribution')                                        
        self.thickness_distribution_Element1.to_excel(writer, sheet_name=f'thickness_distribution_{self.Element1}')
        self.thickness_distribution_Element2.to_excel(writer, sheet_name=f'thickness_distribution_{self.Element2}')
        writer.save()
    
    def draw_figures_1D(self):

        font=12
        plt.figure(0)
        plt.clf()
        data = self.composition_distribution_Element1
        plt.plot(np.linspace(0,self.xsize,self.number_points),data,linewidth=3, linestyle="--")
        plt.title(f"Concentration distribution of {self.Element1}",fontsize=font)
        plt.xlabel("X position [mm]",fontsize=font)
        plt.ylabel(f"{self.Element1} concentration [at.%]",fontsize=font)
        plt.minorticks_on()
        plt.tick_params(axis="both", which = "major", length=10, direction ="in",labelsize=10)
        plt.tick_params(axis="both", which = "minor", length=5, direction ="in")    

        #self.fig = list(map(plt.figure, plt.get_fignums()))

    #optimazation part / model are fitted to the data loaded from xlsx file      
        
    @staticmethod
    def optimization(data,filename_poly,Element1,Element2,total_thickness, xsize, Element3,thickness_Element3):

        mol_data = dict(pd.read_excel("molecular_database.xlsx"))
        polynomial = np.poly1d(np.load(filename_poly))
        number_points = 201
        #total_thickness = 100

        xt = 65
        xt2 = -65            
   
        def concentration_calibration(x,cElement1):
             
            #sympy part
            tElement1, tElement2 = symbols('tElement1, tElement2')
            eq1 = Eq(100*(tElement1/mol_data[Element1])/((tElement1/mol_data[Element1]) + ((tElement2)/mol_data[Element2])+ ((thickness_Element3)/mol_data[Element3])),cElement1)
            eq2 =  Eq(tElement1 + tElement2,total_thickness)
            solved = dict((str(key), float(value)) for (key, value) in solve((eq1,eq2),(tElement1,tElement2)).items())
            thickness_Element1  = solved['tElement1'] 
            thickness_Element2 = solved['tElement2']

            epsilon_Element1 = thickness_Element1/polynomial(65)
            epsilon_Element2 = thickness_Element2/polynomial(65)

            #model calculation
            ro = np.sqrt((x - xt)**2)
            ro2 = np.sqrt((x - xt2)**2)
            thickness_distribution_Element1=epsilon_Element1*polynomial(ro)
            thickness_distribution_Element2=epsilon_Element2*polynomial(ro2)
            composition_distribution_Element1=100*thickness_distribution_Element1/mol_data[Element1][0]/(thickness_distribution_Element1/mol_data[Element1][0]+thickness_distribution_Element2/mol_data[Element2][0]+ ((thickness_Element3)/mol_data[Element3][0]))
            return composition_distribution_Element1
        
        x = np.array(data.index)-xsize/2
        y = np.array(data.iloc[:,0])
        param, param_cov = curve_fit(concentration_calibration, x, y, bounds=(0,100), method ="trf")
        return param[0]

In [3]:
# tab 2
class Concentration_distribution(ttk.Frame):
    
    def __init__(self, concentration):
        super().__init__(concentration)

        self.concentration = concentration
        self.current_value=0

        self.x, self.y = 0.02, 0.05
        self.x2=0.15
        
        self.mol_data = dict(pd.read_excel("molecular_database.xlsx"))
        
        self.create_field()


    def create_field(self):
        
        self.button_calculate = tk.Button(self, text="Calculate", command = self.calculate)
        self.button_calculate.place(relx=self.x, rely=0.8)        
         
        self.text_elements = tk.Label(self, text="Name of elements", font=("", 10))
        self.text_elements.place(relx=self.x, rely=self.y)
        self.combobox_element1 = Combobox(self, values=list(self.mol_data.keys())[1:], state = "readonly",width=5)
        self.combobox_element1.place(relx= self.x2, rely= self.y)

        self.text_sample_size = tk.Label(self, text="Sample dimension [mm]:", font=("", 10))
        self.text_sample_size.place(relx=self.x, rely=self.y+0.1)         
        self.text_xsize = tk.Label(self, text="X size:", font=("", 10))
        self.text_xsize.place(relx=self.x2, rely=self.y+0.07)        
        self.entry_xsize = tk.Entry(self, width=10)
        self.entry_xsize.place(relx= self.x2, rely= self.y+0.1)
        self.text_ysize = tk.Label(self, text="Y size:", font=("", 10))
        self.text_ysize.place(relx=self.x2+0.07, rely=self.y+0.07)         
        self.entry_ysize = tk.Entry(self, width=10)
        self.entry_ysize.place(relx= self.x2+0.07, rely= self.y+0.1)
        self.combobox_element2 = Combobox(self, values=list(self.mol_data.keys())[1:], state = "readonly",width=5)
        self.combobox_element2.place(relx= self.x2+0.05, rely= self.y)
        
        self.text_AK = tk.Label(self, text="A-K height [cm]", font=("", 10))
        self.text_AK.place(relx=self.x, rely=self.y+0.2)
        self.button_AK = tk.Button(self, text = "Load .npy file", command=self.load)
        self.button_AK.place(relx=self.x2, rely=self.y+0.2)
        
        self.text_thickness = tk.Label(self, text="Alloy thickness [A]", font=("", 10))
        self.text_thickness.place(relx=self.x, rely=self.y+0.3)        
        self.entry_thickness = tk.Entry(self, width=10)
        self.entry_thickness.place(relx= self.x2, rely= self.y+0.3)
        
        self.text_concentration = tk.Label(self, text="Concentration [at.%]", font=("", 10))
        self.text_concentration.place(relx=self.x, rely=self.y+0.4)        
        self.entry_concentration = tk.Entry(self, width=10)
        self.entry_concentration.place(relx= self.x2, rely= self.y+0.4)
        
        self.text_slider = tk.Label(self, text="Sample center position", font=("", 10))
        self.text_slider.place(relx=self.x, rely=self.y+0.55)
        self.slider_position = tk.Scale(self, from_=-65, to = 65, orient="horizontal")
        self.slider_position.place(relx=self.x2, rely=self.y+0.525)        

        self.text_layer = tk.Label(self, text="Additional magnetic layer", font=("", 10))
        self.text_layer.place(relx=self.x, rely=self.y+0.65)
        self.combobox_element3 = Combobox(self, values=list(self.mol_data.keys())[1:], state = "readonly",width=5)
        self.combobox_element3.place(relx= self.x2, rely= self.y+0.65)
        self.combobox_element3.current(0)        
        self.text_layer_thickness = tk.Label(self, text="Thickness [A]", font=("", 10))
        self.text_layer_thickness.place(relx=self.x2+0.05, rely=self.y+0.62)        
        self.entry_add_layer = tk.Entry(self, width=10)
        self.entry_add_layer.place(relx= self.x2+0.05, rely= self.y+0.65)
        self.entry_add_layer.insert(0,0)
        
    def load(self):
            self.filename = fd.askopenfilename(filetypes =[('NumPy File', '*.npy')])
            self.text_filename = tk.Label(self, text=f"{self.filename}", font=("", 10))
            self.text_filename.place(relx=0, rely=self.y+0.35)
            
    def save(self):
        filename_save = fd.asksaveasfilename(filetypes=[("Excel file","*.xlsx")], defaultextension = "*.xlsx")        
        self.sample.save_2D(filename_save)
        
    def calculate(self):
        self.sample = concentration(self.combobox_element1.get(),self.combobox_element2.get(),self.entry_concentration.get(),self.entry_thickness.get(),self.filename, self.entry_xsize.get(), self.entry_ysize.get(), self.slider_position.get(), self.combobox_element3.get(),self.entry_add_layer.get())
        self.sample.calculate_concentration_2D()
        self.sample.draw_figures_2D()

        xwidth, ywidth = 400,300
        
        canvas_comp_dist = FigureCanvasTkAgg(figure=self.sample.fig[0], master=self)  # A tk.DrawingArea.
        canvas_comp_dist.draw()
        canvas_comp_dist.get_tk_widget().place(relx=0.3, rely=0.05)
        canvas_comp_dist.get_tk_widget().configure(width=xwidth,height=ywidth)
        
        canvas_comp = FigureCanvasTkAgg(figure=self.sample.fig[1], master=self)  # A tk.DrawingArea.
        canvas_comp.draw()
        canvas_comp.get_tk_widget().place(relx=0.65, rely=0.05)
        canvas_comp.get_tk_widget().configure(width=xwidth,height=ywidth)
        
        canvas_thickness_dist = FigureCanvasTkAgg(figure=self.sample.fig[2], master=self)  # A tk.DrawingArea.
        canvas_thickness_dist.draw()
        canvas_thickness_dist.get_tk_widget().place(relx=0.3, rely=0.5)
        canvas_thickness_dist.get_tk_widget().configure(width=xwidth,height=ywidth)
        
        canvas_thickness = FigureCanvasTkAgg(figure=self.sample.fig[3], master=self)  # A tk.DrawingArea.
        canvas_thickness.draw()
        canvas_thickness.get_tk_widget().place(relx=0.65, rely=0.5)
        canvas_thickness.get_tk_widget().configure(width=xwidth,height=ywidth)         

        self.save_data = tk.Button(self, text="Save data", command = self.save)
        self.save_data.place(relx=self.x, rely=0.9)
        
        self.export_plot = tk.Button(self, text="Export plot", command = self.save_fig)
        self.export_plot.place(relx=self.x, rely=0.95)
        
    def save_fig(self):
        filename_image = fd.asksaveasfilename(filetypes=[("pdf file","*.pdf")], defaultextension = "*.pdf")        
        pdf = PdfPages(filename_image)
        #pdf.savefig(self.fig[0])
        for image in self.sample.fig:
            pdf.savefig(image)
        pdf.close()        

if __name__ == "__main__":
    root = tk.Tk()
    root.title("Concentration_distribution by Łukasz Frąckowiak")
    root.resizable(False, False)
    main = Concentration_distribution(root)
    main.pack(side="top", fill="both", expand=True)
    root.wm_geometry("1200x700")
    root.mainloop()


In [4]:
#tab 1
class Concentration_fit(ttk.Frame):
    
    def __init__(self, concentration):
        super().__init__(concentration)

        self.concentration = concentration       
                
        #cordinations initialization
        self.x, self.y = 0.02, 0.05
        self.x2=0.15
        
        self.mol_data = dict(pd.read_excel("molecular_database.xlsx"))
        
        self.start_frame()
       
    def load_poly(self):
        self.filename_poly = fd.askopenfilename(filetypes =[('NumPy File', '*.npy')])
        self.text_filename_poly = tk.Label(master=self, text=f"{self.filename_poly}", font=("", 10))
        self.text_filename_poly.place(relx=0, rely=self.y+0.35)
   

    def start_frame(self):
        self.text_sample_size = tk.Label(self, text="Sample dimension [mm]:", font=("", 10))
        self.text_sample_size.place(relx=self.x, rely=self.y+0.05)        
        
        self.text_xsize = tk.Label(self, text="X size:", font=("", 10))
        self.text_xsize.place(relx=self.x2, rely=self.y+0.02)
        self.entry_xsize = tk.Entry(self, width=10)
        self.entry_xsize.place(relx= self.x2, rely= self.y+0.05)        
        
        self.button_calculate_2 = tk.Button(master=self, text="Calculate", command = self.calculate_frame)
        self.button_calculate_2.place(relx=self.x+0.1, rely=0.6)
        
        self.text_Elements = tk.Label(master=self, text="Name of elements", font=("", 10))
        self.text_Elements.place(relx=self.x, rely=self.y+0.15)
        self.combobox_Element1 = Combobox(master=self, values=list(self.mol_data.keys())[1:], state = "readonly",width=5)
        self.combobox_Element1.place(relx= self.x2, rely= self.y+0.15)
        
        self.combobox_Element2 = Combobox(master=self, values=list(self.mol_data.keys())[1:], state = "readonly",width=5)
        self.combobox_Element2.place(relx= self.x2+0.05, rely= self.y+0.15)
        
        self.text_AK = tk.Label(master=self, text="Polynomial file", font=("", 10))
        self.text_AK.place(relx=self.x, rely=self.y+0.3)
        self.button_AK = tk.Button(master=self, text = "Load .npy file", command=self.load_poly)
        self.button_AK.place(relx=self.x2, rely=self.y+0.3)
        
        self.text_data = tk.Label(master=self, text="Experimental data", font=("", 10))
        self.text_data.place(relx=self.x, rely=self.y+0.45)
        self.button_AK = tk.Button(master=self, text = "Load excel data file", command=self.load_data)
        self.button_AK.place(relx=self.x2, rely=self.y+0.45)
        
        self.text_alloy_layer = tk.Label(self, text="Alloy thickness [A]", font=("", 10))
        self.text_alloy_layer.place(relx=self.x, rely=self.y+0.75)        
        self.entry_alloy_layer = tk.Entry(self, width=10)
        self.entry_alloy_layer.place(relx= self.x2, rely= self.y+0.75)
        self.entry_alloy_layer.insert(0,100)
        
        self.text_layer = tk.Label(self, text="Additional magnetic layer", font=("", 10))
        self.text_layer.place(relx=self.x, rely=self.y+0.85)
        self.combobox_element3 = Combobox(self, values=list(self.mol_data.keys())[1:], state = "readonly",width=5)
        self.combobox_element3.place(relx= self.x2, rely= self.y+0.85)
        self.combobox_element3.current(0)        
        self.text_layer_thickness = tk.Label(self, text="Thickness [A]", font=("", 10))
        self.text_layer_thickness.place(relx=self.x2+0.05, rely=self.y+0.82)        
        self.entry_add_layer = tk.Entry(self, width=10)
        self.entry_add_layer.place(relx= self.x2+0.05, rely= self.y+0.85)
        self.entry_add_layer.insert(0,0)        
        
    def load_data(self):
        self.filename_data = fd.askopenfilename(filetypes =[('Excel Data File', '*.xlsx')])
        self.text_filename_data = tk.Label(master=self, text=f"{self.filename_data}", font=("", 10))
        self.text_filename_data.place(relx=0, rely=self.y+0.5)
            
    def calculate_frame(self):
        self.save_plot = tk.Button(self, text="Export plot", command = self.save_plot)
        self.save_plot.place(relx=self.x+0.1, rely=0.65)
        
        Element1 = self.combobox_Element1.get()
        Element2 = self.combobox_Element2.get()
        
        data = pd.read_excel(self.filename_data,index_col=[0],header=None)
        value = concentration.optimization(data,self.filename_poly,Element1,Element2,float(self.entry_alloy_layer.get()),float(self.entry_xsize.get()),self.combobox_element3.get(), float(self.entry_add_layer.get()))
        self.sample_2 = concentration(Element1, Element2, value, 100, self.filename_poly,float(self.entry_xsize.get()),20,0,self.combobox_element3.get(), float(self.entry_add_layer.get()))
        self.sample_2.calculate_concentration_1D()
        self.sample_2.draw_figures_1D()

        font=12
        plt.figure(0)
        plt.scatter(data.index,data.iloc[:,0], c="r", s=50)
        plt.title(f"Concentration distribution of {Element1}",fontsize=font)
        plt.xlabel("X position [mm]",fontsize=font)
        plt.ylabel(f"{Element1} concentration [at.%]",fontsize=font)
        plt.minorticks_on()
        plt.tick_params(axis="both", which = "major", length=10, direction ="in",labelsize=10)
        plt.tick_params(axis="both", which = "minor", length=5, direction ="in")
        plt.legend([f"Model fit for c{Element1} = {round(value,2)} at.%","Experimental data"],prop={'size': 20})
        
        self.fig = list(map(plt.figure, plt.get_fignums()))
        canvas_comp = FigureCanvasTkAgg(master=self, figure=self.fig[0]) #=self.sample_2fig[0])  # A tk.DrawingArea.
        canvas_comp.draw()
        canvas_comp.get_tk_widget().place(relx=0.3, rely=0.05)
        canvas_comp.get_tk_widget().configure(width=800,height=600)

    def save_plot(self):
        filename_image = fd.asksaveasfilename(filetypes=[("pdf file","*.pdf")], defaultextension = "*.pdf")        
        pdf = PdfPages(filename_image)
        pdf.savefig(self.fig[0])
        pdf.close()
        self.text_export = tk.Label(self, text=f"Plot saved in {filename_image}", font=("", 10))
        self.text_export.place(relx=0, rely=0.7)        
    
if __name__ == "__main__":
    root = tk.Tk()
    root.title("Concentration_fit by Łukasz Frąckowiak")
    root.resizable(False, False)
    main = Concentration_fit(root)
    main.pack(side="top", fill="both", expand=True)
    root.wm_geometry("1200x700")
    root.mainloop()


## Part related to numerical deposition calculation

In [5]:
class Numerical_deposition(ttk.Frame):
    
    def __init__(self):
        super().__init__()

        self.x, self.y = 0.05, 0.1 
        self.degree = 20
        self.count=0
        self.start_frame()
        self.create_field()
             
    def start_frame(self):
        self.text = tk.Label(self, text="Please enter all necesary parameters of setup", font=("", 10))
        self.text.place(relx=0, rely=0)
        
        self.button_calculate = tk.Button(self, text="Calculate", command = self.calculate)
        self.button_calculate.place(relx= self.x, rely= self.y+0.7)


    def create_field(self):

        self.label_poly_degree = tk.Label(self, text="Polynomial degree", font=("",10))
        self.label_poly_degree.place(relx=0.5, rely=0.1)
        self.entry_poly_degree = tk.Entry(self)
        self.entry_poly_degree.insert(0,40)
        self.entry_poly_degree.place(relx=0.6, rely=0.1)
        
        self.text_TargetOuterRadius = tk.Label(self, text="Target outer radius [mm]", font=("", 10))
        self.text_TargetOuterRadius.place(relx=self.x, rely=self.y+0.05)
        self.entry_TargetOuterRadius = tk.Entry(self, width=10)
        self.entry_TargetOuterRadius.insert(0,25)
        self.entry_TargetOuterRadius.place(relx= self.x, rely= self.y+0.1)
        
        self.text_TargetInnerRadius = tk.Label(self, text="Target inner radius [mm]", font=("", 10))
        self.text_TargetInnerRadius.place(relx=self.x, rely=self.y+0.15)
        self.entry_TargetInnerRadius = tk.Entry(self, width=10)
        self.entry_TargetInnerRadius.insert(0,10)
        self.entry_TargetInnerRadius.place(relx= self.x, rely= self.y+0.2)

        self.text_TargetCellNumber = tk.Label(self, text="Number of target cells", font=("", 10))
        self.text_TargetCellNumber.place(relx=self.x, rely=self.y+0.25)
        self.entry_TargetCellNumber = tk.Entry(self, width=10)
        self.entry_TargetCellNumber.insert(0,50)
        self.entry_TargetCellNumber.place(relx= self.x, rely= self.y+0.3)        
        
        self.text_AK_Height = tk.Label(self, text="A-K height [mm]", font=("", 10))
        self.text_AK_Height.place(relx=self.x, rely=self.y+0.35)
        self.entry_AK_Height = tk.Entry(self, width=10)
        self.entry_AK_Height.insert(0,97)
        self.entry_AK_Height.place(relx= self.x, rely= self.y+0.4)

        self.text_ChimneyHeight = tk.Label(self, text="Chimney height [mm]", font=("", 10))
        self.text_ChimneyHeight.place(relx=self.x, rely=self.y+0.45)
        self.entry_ChimneyHeight = tk.Entry(self, width=10)
        self.entry_ChimneyHeight.insert(0,30)
        self.entry_ChimneyHeight.place(relx= self.x, rely= self.y+0.5)

        self.text_ChimneyRadius = tk.Label(self, text="Chimney radius [mm]", font=("", 10))
        self.text_ChimneyRadius.place(relx=self.x, rely=self.y+0.55)
        self.entry_ChimneyRadius = tk.Entry(self, width=10)
        self.entry_ChimneyRadius.insert(0,25)
        self.entry_ChimneyRadius.place(relx= self.x, rely= self.y+0.6)            
        
    
    def variables(self):
        self.TargetOuterRadius = float(self.entry_TargetOuterRadius.get())
        self.TargetInnerRadius = float(self.entry_TargetInnerRadius.get())
        self.TargetCellNumber = int(self.entry_TargetCellNumber.get())
        self.AK_Height = float(self.entry_AK_Height.get())
        self.ChimneyHeight = float(self.entry_ChimneyHeight.get())
        self.ChimneyRadius = float(self.entry_ChimneyRadius.get())
        self.PlaneSize = float(500)
        
        self.XStartingTargetPosition = -self.TargetOuterRadius/2
        self.XEndingTargetPosition = self.TargetOuterRadius/2
        self.YStartingTargetPosition = -self.TargetOuterRadius/2
        self.YEndingTargetPosition = self.TargetOuterRadius/2

        self.XTargetCellSize = (self.XEndingTargetPosition - self.XStartingTargetPosition)/self.TargetCellNumber
        self.YTargetCellSize = (self.XEndingTargetPosition - self.XStartingTargetPosition)/self.TargetCellNumber
        self.YTargetCenterPosition = 0
        self.XTargetCenterPosition = 0

        self.StartingPlanePosition = -self.PlaneSize/2
        self.EndingPlanePosition = self.PlaneSize/2

    def distribution(self):
        
        def ChimneyIntersect(x1,y1,x2,y2,h,R):
            value = (-h*((-2)*x1*x2 + 2*x2**2 - 2*y1*y2 + 2*y2**2 + np.sqrt((2*x1*x2 - 2*x2**2 + 2*y1*y2 - 2*y2**2)**2 - 4*(-R**2 + x2**2 + y2**2)*(x1**2 - 2*x1*x2 + x2**2 + y1**2 - 2*y1*y2 + y2**2)))/(2*(x1**2 - 2*x1*x2 + x2**2 + y1**2 - 2*y1*y2 + y2**2)))
            return value
        
        XCount = 0
        g = np.zeros(501)
        XPlane = self.StartingPlanePosition
        while XPlane <= self.EndingPlanePosition: 
            XTarget = self.XStartingTargetPosition
            while XTarget <= self.XEndingTargetPosition:
                YTarget = self.YStartingTargetPosition
                while YTarget <= self.YEndingTargetPosition:

                    TargetShape = np.sqrt(XTarget**2+YTarget**2)
                    PlaneTargetCellsDistance = np.sqrt((XTarget-XPlane)**2+YTarget**2)

                    if self.TargetInnerRadius <= TargetShape <= self.TargetOuterRadius:

                        if self.ChimneyHeight != 0:

                            if PlaneTargetCellsDistance > self.TargetOuterRadius:

                                    if ChimneyIntersect(XPlane,0,XTarget,YTarget,self.AK_Height,self.ChimneyRadius) < -self.ChimneyHeight:
                                        g[XCount] += (self.AK_Height**2)/((self.AK_Height**2 + PlaneTargetCellsDistance**2)**2)
                                    else: pass

                            else: g[XCount] += (self.AK_Height**2)/((self.AK_Height**2 + PlaneTargetCellsDistance**2)**2) 

                        else: g[XCount] += (self.AK_Height**2)/((self.AK_Height**2 + PlaneTargetCellsDistance**2)**2)  

                    else: pass    


                    YTarget += self.YTargetCellSize
                XTarget += self.XTargetCellSize
            XPlane += 1
            XCount += 1               
        self.g = g

    def fit(self):
        plt.cla()
        self.g = pd.DataFrame(self.g)
        self.g.index = np.arange(-250,251,1)
        self.G = np.poly1d(np.polyfit(self.g.index[100:-100],self.g[0][100:-100],int(self.entry_poly_degree.get())))
        plt.scatter(self.g.index,self.g[0])
        plt.plot(self.g.index[100:-100],self.G(self.g.index[100:-100]), "r",linewidth=3)
        plt.ylabel(r"G $[mm^{-2}]$")
        plt.xlabel("Plane position [mm]")
        plt.legend(labels=["Numerical data","Polynomial fit"])
        self.fig = list(map(plt.figure, plt.get_fignums()))

    def calculate(self):

        self.variables()
        self.distribution()
        self.fit()
        self.button_save = tk.Button(self, text="Export polynomial", command = self.export_poly)
        self.button_save.place(relx=0.3, rely=0.1)
        
        self.button_export = tk.Button(self, text="Export plot", command = self.export)
        self.button_export.place(relx=0.3, rely=0.15)
        
        self.button_save_data = tk.Button(self, text="Save data", command = self.save_data)
        self.button_save_data.place(relx=0.3, rely=0.2)
        
            
        canvas = FigureCanvasTkAgg(figure = self.fig[0], master = self)  # A tk.DrawingArea.
        canvas.draw()
        canvas.get_tk_widget().place(relx=0.3, rely=0.25)
        canvas.get_tk_widget().configure(width=700,height=500)

    def export_poly(self):
        filename_poly = fd.asksaveasfilename(filetypes=[("Npy file","*.npy")], defaultextension = "*.npy")
        np.save(filename_poly, self.G)        #f"polynomial_AK_{int(self.AK_Height)}.npy"
        self.text_export = tk.Label(master = self, text=f"Polynomial saved in {filename_poly}", font=("", 10))
        self.text_export.place(relx=0.4, rely=0.1)
        
        self.g.to_excel("G_data.xlsx")

    def export(self):
        filename_image = fd.asksaveasfilename(filetypes=[("pdf file","*.pdf")], defaultextension = "*.pdf")        
        pdf = PdfPages(filename_image)
        pdf.savefig(self.fig[0])
        pdf.close()
        self.text_export = tk.Label(self, text=f"Plot saved in {filename_image}", font=("", 10))
        self.text_export.place(relx=0.4, rely=0.15)
        
    def save_data(self):
        filename_save = fd.asksaveasfilename(filetypes=[("Excel file","*.xlsx")], defaultextension = "*.xlsx")              
        writer = pd.ExcelWriter(path = filename_save, engine='xlsxwriter')
        self.g.to_excel(writer, sheet_name='G_distribution')
        writer.save()
           

if __name__ == "__main__":
    root = tk.Tk()
    root.title("Numercal deposition by Łukasz Frąckowiak")
    root.resizable(False, False)
    main = Numerical_deposition()
    main.pack(side="top", fill="both", expand=True)
    root.wm_geometry("1200x700")
    root.mainloop()


## Parts related to calibration of sputtering parameters

In [6]:
class point:
    "Power_Element1 - power suply of Element1 target, Power_Element2 - power suply of Element2 target, Concentration_Element1 - concentrations in the middle of sample, total_thickness - total thickness of alloy"
    def __init__(self, Element1, Element2, Power_Element1, Power_Element2, Concentration_Element1, total_thickness, time):
        
        #data = {"Co":6.67*10**-6, "Ni":6.59*10**-6, "Au":10.21*10**-6, "Pt":9.09*10**-6, "Tb":1.93*10**-5} # dict with molecular volume of elements
        data = dict(pd.read_excel("molecular_database.xlsx"))
        
        self.Element1 = str(Element1)
        self.Element2 = str(Element2)
        self.Power_Element1 = float(Power_Element1)
        self.Power_Element2 = float(Power_Element2)
        self.Concentration_Element1 = float(Concentration_Element1)
        self.total_thickness = float(total_thickness)
        self.time = int(time)
        
        #inicjalizacja wartości stałych
        self.VElement1 = float(data[str(Element1)])
        self.VElement2 = float(data[str(Element2)])
        self.deposition_rate()
        
       
    def thickness(self):
        tElement11, tElement21 = symbols('tElement1, tElement2')
        eq1 = Eq(100*(tElement11/self.VElement1)/((tElement11/self.VElement1) + ((tElement21)/self.VElement2)),self.Concentration_Element1)
        eq2 =  Eq(tElement21 + tElement11,self.total_thickness)
        solved = dict((str(key), float(value)) for (key, value) in solve((eq1,eq2),(tElement11,tElement21)).items())
        self.tElement1  = solved['tElement1'] 
        self.tElement2 = solved['tElement2']
        return solved
    
    def deposition_rate(self):
        try:
            dr_solved = dict((key, value/self.time) for (key, value) in self.thickness().items())
            self.rate_Element1 = dr_solved['tElement1']
            self.rate_Element2 = dr_solved['tElement2']
            return dr_solved
        except ZeroDivisionError as Error:
            print("Time Error")
        

    @staticmethod
    def line(*samples):
        plt.clf()
        line_param={"Element":[],"Slope":[],"Intercept":[]}
        x = np.linspace(0,70,100)
        #firslty we do list comprehension, after that to flat array to 1D we use sum(list,[]) method, next we transform list to set to get only unique value, and as last we create empty dictionary by another list of comprehension
        element_dict = {i: [] for i in sum([[element.Element1, element.Element2] for element in samples],[])}
        power_dict = copy.deepcopy(element_dict)
        for sample in samples:
            element_dict[sample.Element1].append(sample.rate_Element1)
            element_dict[sample.Element2].append(sample.rate_Element2)
            power_dict[sample.Element1].append(sample.Power_Element1)
            power_dict[sample.Element2].append(sample.Power_Element2)

        element_dict[sample.Element1].append(0)
        element_dict[sample.Element2].append(0)
        power_dict[sample.Element1].append(0)
        power_dict[sample.Element2].append(0)             
            
        for element in element_dict:
            fit = np.polyfit(x=power_dict[element], y=element_dict[element], deg=1)
            fit1D = np.poly1d(fit)

            line_param["Element"].append(element)
            line_param["Slope"].append(fit[0])
            line_param["Intercept"].append(fit[1])
            
            
            plt.scatter(x=power_dict[element], y=element_dict[element])
            plt.legend(sum([[key]*2 for key in element_dict.keys()],[])) # double keys, in short a,b keys transform into a,a,b,b
            plt.xlabel("Power of target supply [W]")
            plt.ylabel("Sputtering rate [A/s]")
            
            plt.plot(x,fit1D(x))
            #plt.rcParams["figure.figsize"] = (10,10)
            
        figs = list(map(plt.figure, plt.get_fignums()))
        return line_param, figs

    
    @staticmethod
    def save(*samples, filename):
        data, fig = point.line(*samples)
        data = pd.DataFrame(data)
        # here if file doesnt exist. all calculated data will be saved into .xlsx file
        try:
            old = pd.read_excel(filename,index_col=[0])
            for element in data["Element"]:
                # here we check if old data contains information about each element calculated today if not it will add new row in data, if contains it will update information
                try:
                    test = old.loc[old["Element"]==element,"Slope"].item() # chceck procedure if fail will skip to excpet procedure
                    old.loc[old["Element"]==element,"Slope"]=data.loc[data["Element"]==element,"Slope"].item()
                    old.loc[old["Element"]==element,"Intercept"]=data.loc[data["Element"]==element,"Slope"].item()
                except ValueError:
                    old = pd.concat((old,data.loc[data["Element"]==element]),axis=0).reset_index(drop=True)
            old.to_excel(filename)        
        except FileNotFoundError:
            pd.DataFrame(data).to_excel(filename)
            

In [7]:
class Sputtering_calibration(ttk.Frame):
    
    def __init__(self, point):
        super().__init__(point)
        
        self.point = point
        #cordinations initialization
        self.x, self.y = 0.2, 0.1 

        self.start_frame()
        self.create_field()
     
    def start_frame(self):
        self.text = tk.Label(self, text="Please enter: \nname of 1st element and 2nd element,\n Power of element_1 [W], Power of element_2 [W], \nconcentration of element_1 in the center of sample [at.%], \ntotal alloy thickness [A], sputtering time [s] \n e.g. Co, Au, 10, 20, 12, 80, 100", font=("", 10))
        self.text.place(relx=0, rely=0,)
        
        self.button_calculate = tk.Button(self, text="Calculate", command = self.calculate)
        self.button_calculate.place(relx=0.08, rely=0.7)

    def create_field(self):
        
        self.entry1 = tk.Entry(self, width=30)
        self.entry1.place(relx= self.x-0.15, rely= self.y+0.105)
        
        self.entry2 = tk.Entry(self, width=30)
        self.entry2.place(relx= self.x-0.15, rely= self.y+0.155)
        
        self.entry3 = tk.Entry(self, width=30)
        self.entry3.place(relx= self.x-0.15, rely= self.y+0.205)
        
        self.entry4 = tk.Entry(self, width=30)
        self.entry4.place(relx= self.x-0.15, rely= self.y+0.255)
        
        self.entry5 = tk.Entry(self, width=30)
        self.entry5.place(relx= self.x-0.15, rely= self.y+0.305)
        
        self.entry6 = tk.Entry(self, width=30)
        self.entry6.place(relx= self.x-0.15, rely= self.y+0.355)
        
        self.entry7 = tk.Entry(self, width=30)
        self.entry7.place(relx= self.x-0.15, rely= self.y+0.405)
        
        self.entry8 = tk.Entry(self, width=30)
        self.entry8.place(relx= self.x-0.15, rely= self.y+0.455)
        
        self.entry9 = tk.Entry(self, width=30)
        self.entry9.place(relx= self.x-0.15, rely= self.y+0.505)
        
        self.entry10 = tk.Entry(self, width=30)
        self.entry10.place(relx= self.x-0.15, rely= self.y+0.555)

    def calculate(self):
        
        self.button_save = tk.Button(self, text="SAVE", command = self.save)
        self.button_save.place(relx=0.08, rely=0.85)
        
        self.button_export = tk.Button(self, text="Export plot", command = self.export)
        self.button_export.place(relx=0.08, rely=0.9)
        

        
        def transform(entryWidget):
            lista=[]
            for entry in entryWidget.replace(" ","").split(sep=","):
                try:
                    lista.append(float(entry))
                except ValueError:
                    lista.append(str(entry))
            return lista        
            
        point_list = []
        entry_list = map(transform,[self.entry1.get(),self.entry2.get(),self.entry3.get(),self.entry4.get(),self.entry5.get(),self.entry6.get(),self.entry7.get(),self.entry8.get(),self.entry9.get(),self.entry10.get()])
        
        for entry in entry_list:
            try:
                 point_list.append(point(*entry))
            except KeyError:
                messagebox.showerror("Error 1", "At least one of element does not appear in database.\nPlease check spelling or fill molecular_database.xlsx entry.")
            except TypeError:
                if len(entry)!=1:
                    messagebox.showerror("Error 2", "You input wrong number of parameters.\nPlease correct entry values")
            except ValueError:
                messagebox.showerror("Error 3", "Please correct entry values")
        self.point_list = point_list        
        data_points, self.fig = point.line(*point_list)
        self.lists_of_points = pd.DataFrame(data_points)    
        self.text2 = tk.Label(self, text=f"{self.lists_of_points}", font=("", 10))
        self.text2.place(relx=0.5, rely=0.02)
        
        canvas = FigureCanvasTkAgg(figure=self.fig[0], master = self)  # A tk.DrawingArea.
        canvas.draw()
        canvas.get_tk_widget().place(relx=0.3, rely=0.15)
        canvas.get_tk_widget().configure(width=800,height=550)
        

    def save(self):
        filename_xlsx = fd.asksaveasfilename(filetypes=[("Excel file","*.xlsx")], defaultextension = "*.xlsx")         
        point.save(*self.point_list, filename=filename_xlsx)

    def export(self):
        filename_image = fd.asksaveasfilename(filetypes=[("pdf file","*.pdf")], defaultextension = "*.pdf")        
        pdf = PdfPages(filename_image)
        pdf.savefig(self.fig[0])
        pdf.close()

if __name__ == "__main__":
    root = tk.Tk()
    root.title("Sputtering calibration by Łukasz Frąckowiak")
    root.resizable(False, False)
    main = Sputtering_calibration(root)
    main.pack(side="top", fill="both", expand=True)
    root.wm_geometry("1200x700")
    root.mainloop()


## Parts related to caluculation of targets powers (sputtering rates)

In [8]:
class sample:
    def __init__(self, Element1, Element2, concentration_Element1, total_thickness, sputtering_time):

        self.mol_data = dict(pd.read_excel("molecular_database.xlsx")) # dict with molecular volume of elements
        
        self.Element1 = str(Element1)
        self.Element2 = str(Element2)
        self.concentration_Element1 = float(concentration_Element1)
        self.total_thickness = float(total_thickness)
        self.sputtering_time = int(sputtering_time)
        
    def load_sputtering_parameters(self,sputtering_parameters):
        self.sputtering_parameters = pd.read_excel(sputtering_parameters,index_col=[0])
            
    def power_calculation(self,AK_Value):
        self.load_sputtering_parameters(AK_Value)
        tElement1, tElement2 = symbols('tElement1, tElement2')
        eq1 = Eq(100*(tElement1/self.mol_data[self.Element1])/((tElement1/self.mol_data[self.Element1]) + ((tElement2)/self.mol_data[self.Element2])),self.concentration_Element1)
        eq2 =  Eq(tElement2 + tElement1,self.total_thickness)
        solved = dict((str(key), float(value)) for (key, value) in solve((eq1,eq2),(tElement1,tElement2)).items())
        self.thickness_Element1  = solved['tElement1'] 
        self.thickness_Element2 = solved['tElement2']
        self.sputtering_rate_Element1 = float(self.thickness_Element1/self.sputtering_time)
        self.sputtering_rate_Element2 = float(self.thickness_Element2/self.sputtering_time)
        df = self.sputtering_parameters
        try:
            self.power_Element1 = (self.sputtering_rate_Element1-df.loc[df["Element"]==self.Element1,"Intercept"].item())/df.loc[df["Element"]==self.Element1,"Slope"].item()
            self.power_Element2 = (self.sputtering_rate_Element2-df.loc[df["Element"]==self.Element2,"Intercept"].item())/df.loc[df["Element"]==self.Element2,"Slope"].item()
        except ValueError:
            print("Please provide sputtering parameters for specific elements")


In [9]:
class Sputtering_rate(ttk.Frame):
    
    def __init__(self, sample):
        super().__init__(sample)

        self.sample = sample
        self.x, self.y = 0.02, 0.05
        self.x2=0.2
        self.mol_data = dict(pd.read_excel("molecular_database.xlsx"))
        self.create_field()              
     
    def create_field(self):
        self.button_calculate = tk.Button(self, text="Calculate", command = self.calculate)
        self.button_calculate.place(relx=0.6, rely=self.y)        
        
        self.text_elements = tk.Label(self, text="Name of elements", font=("", 10))
        self.text_elements.place(relx=self.x, rely=self.y+0.15)
        self.combobox_element1 = Combobox(self, values=list(self.mol_data.keys())[1:], state = "readonly",width=5)
        self.combobox_element1.place(relx= self.x2, rely= self.y+0.15)
        
        self.combobox_element2 = Combobox(self, values=list(self.mol_data.keys())[1:], state = "readonly",width=5)
        self.combobox_element2.place(relx= self.x2+0.1, rely= self.y+0.15)
        
        self.text_AK = tk.Label(self, text="Sputtering rate file", font=("", 10))
        self.text_AK.place(relx=self.x, rely=self.y+0.3)
        
        self.button_parameter = tk.Button(self, text="Load file",command= self.load_sputtering_parameters)
        self.button_parameter.place(relx=self.x2, rely=self.y+0.3)

        self.text_thickness = tk.Label(self, text="Alloy thickness [A]", font=("", 10))
        self.text_thickness.place(relx=self.x, rely=self.y+0.45)        
        self.entry_thickness = tk.Entry(self)
        self.entry_thickness.place(relx= self.x2, rely= self.y+0.45)
        
        self.text_concentration = tk.Label(self, text="Concentration [at.%]", font=("", 10))
        self.text_concentration.place(relx=self.x, rely=self.y+0.6)        
        self.entry_concentration = tk.Entry(self)
        self.entry_concentration.place(relx= self.x2, rely= self.y+0.6)
        
        self.text_time = tk.Label(self, text="Sputtering time [s]", font=("", 10))
        self.text_time.place(relx=self.x, rely=self.y+0.75)        
        self.entry_time = tk.Entry(self)
        self.entry_time.place(relx= self.x2, rely= self.y+0.75)
    
    def load_sputtering_parameters(self):
        self.filename = fd.askopenfilename(filetypes =[('Parameters Excel File', ['*.xlsx',"*.xls"])])
        self.text_filename = tk.Label(self, text=f"{self.filename}", font=("", 10))
        self.text_filename.place(relx=0, rely=self.y+0.35)        
        
    def calculate(self):
        try:
            Sample = sample(str(self.combobox_element1.get()),str(self.combobox_element2.get()),float(self.entry_concentration.get()),float(self.entry_thickness.get()),int(self.entry_time.get()))
        except ValueError:
            messagebox.showerror("Error 1", "You put wrong parameters to calculation")


        try:
            Sample.power_calculation(self.filename)
        except AttributeError:
             messagebox.showerror("Error 2", "Please provide molecular_database.xlsx file")            
        except FileNotFoundError:
             messagebox.showerror("Error 3", f"Please check if sputtering_parameters excel file exists")

        try:
            self.text_power_Element1 = tk.Label(self, text=f"Power of {Sample.Element1} = {Sample.power_Element1} [W]", font=("", 10))
            self.text_power_Element1.place(relx=0.6, rely=0.6)
            self.text_power_Element2 = tk.Label(self, text=f"Power of {Sample.Element2} = {Sample.power_Element2} [W]", font=("", 10))
            self.text_power_Element2.place(relx=0.6, rely=0.7)
        except AttributeError:
            messagebox.showerror("Error 4", "Please provide sputtering parameters for chosen elements")            


"""
if __name__ == "__main__":
    root = tk.Tk()
    root.title("Sputtering rate by Łukasz Frąckowiak")
    root.resizable(False, False)
    main = Sputtering_rate(root)
    main.pack(side="top", fill="both", expand=True)
    root.wm_geometry("1200x700")
    root.mainloop()
"""   

'\nif __name__ == "__main__":\n    root = tk.Tk()\n    root.title("Sputtering rate by Łukasz Frąckowiak")\n    root.resizable(False, False)\n    main = Sputtering_rate(root)\n    main.pack(side="top", fill="both", expand=True)\n    root.wm_geometry("1200x700")\n    root.mainloop()\n'

## Main window

In [ ]:
class App_window(ttk.Frame):
    
    def __init__(self):
        super().__init__()

        self.columnconfigure(0, weight=1)
        self.rowconfigure(0, weight=1)

        self.nb = ttk.Notebook()
        self.t1 = Concentration_fit(self.nb)        
        self.t2 = Concentration_distribution(self.nb)
        self.t3 = Sputtering_rate(self.nb)
        self.t4 = Sputtering_calibration(self.nb)        
        self.t5 = Numerical_deposition()


        

        self.nb.add(self.t1, text='Concentration fit')
        self.nb.add(self.t2, text='Concentration distribution')
        self.nb.add(self.t3, text='Sputtering rates')
        self.nb.add(self.t4, text='Sputtering calibration')        
        self.nb.add(self.t5, text='Numerical distribution')


        self.nb.pack(fill="both", expand=1)

        
if __name__ == "__main__":
    root = tk.Tk()
    root.title("Alloy structures calculation by Łukasz Frąckowiak")
    root.resizable(False, False)
    main = App_window()
    #main.pack(side="top", fill="both", expand=True)
    root.wm_geometry("1200x700")
    root.mainloop()      